In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

#Delete irrelevant columns
df = df.drop(['RowNumber','CustomerId','Surname'],axis =1)

#Label Encode the categorical columns
label_encoder = LabelEncoder()
df['Geography'] = label_encoder.fit_transform(df['Geography'])
df['Gender'] = label_encoder.fit_transform(df['Gender'])

#Split dataset into X and Y (target variable)(Y) and features(X)
x = df.drop('Exited',axis = 1)
y = df['Exited']

#Split the dataset into train,validation and testing sets(70:15:15)
x_train,x_temp,y_train,y_temp = train_test_split(x,y,test_size = 0.3,random_state=42)
x_val,x_test,y_val,y_test = train_test_split(x_temp,y_temp,test_size = 0.5,random_state = 42)

#Standardize numerical features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.fit_transform(x_val)
x_test = scaler.fit_transform(x_test)




In [ ]:
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score


# Define a function to create, compile, and train the model
def build_train_model(hidden_layers, units_per_layer, epochs, activation, regularization, learning_rate):
    model = keras.Sequential()

    # Add hidden layers with specified units and activation functions
    for i in range(hidden_layers):
        model.add(keras.layers.Dense(units_per_layer[i], activation=activation))

    # Add output layer
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(x_val, y_val), verbose=0)

    # Evaluate the model on the validation set
    y_pred = (model.predict(X_val) > 0.5).astype("int32")

    # Calculate metrics
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)

    return accuracy, precision, recall, history

# Experiment setups
experiments = [
    {'hidden_layers': 1, 'units_per_layer': [128], 'epochs': 30, 'activation': 'sigmoid', 'regularization': None, 'learning_rate': 0.01},
    {'hidden_layers': 2, 'units_per_layer': [128, 64], 'epochs': 20, 'activation': 'sigmoid', 'regularization': None, 'learning_rate': 0.01},
    {'hidden_layers': 3, 'units_per_layer': [128, 64, 32], 'epochs': 10, 'activation': 'sigmoid', 'regularization': None, 'learning_rate': 0.01},
    # Add other experiment setups
]

# Run experiments
results = []
for exp in experiments:
    acc, prec, rec, hist = build_train_model(exp['hidden_layers'], exp['units_per_layer'], exp['epochs'], exp['activation'], exp['regularization'], exp['learning_rate'])
    results.append({'Hidden Layers': exp['hidden_layers'], 'Units per Layer': exp['units_per_layer'], 'Epochs': exp['epochs'],
                    'Activation': exp['activation'], 'Regularization': exp['regularization'], 'Learning Rate': exp['learning_rate'],
                    'Accuracy': acc, 'Precision': prec, 'Recall': rec, 'History': hist})

# Tabulate the results
tabulated_results = pd.DataFrame(results)
print(tabulated_results)


47/47 [==============================] - 0s 814us/step
   Hidden Layers Units per Layer  Epochs Activation Regularization  \
0              1           [128]      30    sigmoid           None   
1              2       [128, 64]      20    sigmoid           None   
2              3   [128, 64, 32]      10    sigmoid           None   

   Learning Rate  Accuracy  Precision    Recall  \
0           0.01  0.870667   0.725000  0.510563   
1           0.01  0.874000   0.810458  0.436620   
2           0.01  0.859333   0.683417  0.478873   

                                             History  
0  <keras.src.callbacks.History object at 0x00000...  
1  <keras.src.callbacks.History object at 0x00000...  
2  <keras.src.callbacks.History object at 0x00000...  


In [ ]:
from easyAI import TwoPlayersGame, Human_Player, AI_Player, Negamax, IDNegamax, DUAL


class TicTacToe(TwoPlayersGame):
    def __init__(self):
        # Set up the game board
        self.board = [0] * 9
        # Define the players
        players = [Human_Player("Player"), AI_Player(Negamax(6), name="AI")]
        # Initialize the TwoPlayersGame class
        TwoPlayersGame.__init__(self, players)

    def possible_moves(self):
        return [i + 1 for i, mark in enumerate(self.board) if mark == 0]

    def make_move(self, move):
        self.board[int(move) - 1] = self.player

    def unmake_move(self, move):
        self.board[int(move) - 1] = 0

    def lose(self):
        # Check if the opponent (next player) has won
        return any(sum(self.board[i + j] == self.opponent for j in (0, 3, 6)) == 3 for i in (0, 1, 2)) \
               or any(sum(self.board[i + 3 * j] == self.opponent for j in (0, 1, 2)) == 3 for i in (0, 1, 2)) \
               or sum(self.board[i] == self.opponent for i in (0, 4, 8)) == 3 \
               or sum(self.board[i] == self.opponent for i in (2, 4, 6)) == 3

    def is_over(self):
        # The game is over if there are no more possible moves or if someone has won
        return (self.possible_moves() == []) or self.lose()

    def show(self):
        # Display the current state of the board
        print("\n" + "\n".join(
            [" ".join([[".", "O", "X"][self.board[3 * j + i]] for i in range(3)]) for j in range(3)]))

    def scoring(self):
        # Scoring function for the AI player
        return -100 if self.lose() else 0


def play_game():
    # Create the Tic-Tac-Toe game instance
    game = TicTacToe()
    # Play the game
    game.play()


def solve_game_with_algorithms():
    # Create the Tic-Tac-Toe game instance
    game = TicTacToe()

    # Solve the game using Iterative Deepening
    print("\nSolving with Iterative Deepening:")
    algo_id = IDNegamax(7, win_score=1000, tt=1)
    result_id, move_id = algo_id(game)
    print(f"Result: {result_id}, Move: {move_id}")

    # Solve the game using Depth-First Search
    print("\nSolving with Depth-First Search:")
    algo_dfs = DUAL(depth=6)
    result_dfs, move_dfs = algo_dfs(game)
    print(f"Result: {result_dfs}, Move: {move_dfs}")


if __name__ == "__main__":
    # Play the game with a human player
    play_game()

    # Solve the game using different algorithms
    solve_game_with_algorithms()
